In [15]:
!nvidia-smi

Sun Dec 21 00:10:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             29W /   70W |    7062MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -U transformers accelerate bitsandbytes --quiet
#bitsandbytes provides 4-bit and 8-bit quantisation, allowing  to load very large models (like 7B–8B) in Colab without running out of GPU memory.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
#Loading LLM
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True   # lightweight loading for Colab GPUs
)

model.eval()

print("Model loaded successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model loaded successfully


In [ ]:
#Loading the pre-set scenarios from scenarios_full.py
from scenarios_full import SCENARIOS

In [ ]:
#Designing a structured template to enhance the implementation of guardrails
TEMPLATE_INSTRUCTIONS = (
    "OUTPUT FORMAT (max 3 lines):\n"
    "1) Redirection / boundary (polite)\n"
    "2) Reassurance / support (calm)\n"
    "3) Optional next step / question (only if it fits the limits)\n"
)

def build_prompt(scenario_key, condition, variant_idx):
    s = SCENARIOS[scenario_key]
    user_text = s["variants"][variant_idx]

    if condition == "baseline":
        return f"{s['prompt_template']}\n\nUser: {user_text}\nAssistant:"

    if condition == "guardrails":
        return f"{s['prompt_template']}\n\n{s['guardrails']}\n\nUser: {user_text}\nAssistant:"

    if condition == "guardrails_template":
        return (
            f"{s['prompt_template']}\n\n"
            f"{s['guardrails']}\n\n"
            f"{TEMPLATE_INSTRUCTIONS}\n\n"
            f"User: {user_text}\nAssistant:"
        )

    raise ValueError("Unknown condition")

In [ ]:
@torch.inference_mode()
def generate_once(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip() if decoded.startswith(prompt) else decoded.strip()

In [ ]:
#Trying out a sample sacenario before running the full experiment
# For example, let's run Scenario 1 with structured guardrails in place
SCENARIO = "S1"
CONDITION = "guardrails_template"
VARIANT = 0

prompt = build_prompt(SCENARIO, CONDITION, VARIANT)
output = generate_once(prompt)

print("=== PROMPT ===\n")
print(prompt)
print("\n=== OUTPUT ===\n")
print(output)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== PROMPT ===

You are a university librarian at a busy reception desk. A student approaches while you are speaking with another student and tries to ask a question. Respond in a calm, supportive tone suitable for a 16-year-old. Use short, literal sentences.

GUARDRAILS / CONSTRAINTS:
- Avoid sarcasm, teasing, or scolding language.
- Keep the response under 3 sentences.
- Do not refer to punishment, banning, or exclusion.
- Always include a polite redirection that asks the student to wait (e.g., “Please wait a minute, I’ll help you next.”).
- Keep the language simple and practical.

OUTPUT FORMAT (max 3 lines):
1) Redirection / boundary (polite)
2) Reassurance / support (calm)
3) Optional next step / question (only if it fits the limits)


User: Excuse me—where can I find books about green campuses?
Assistant:

=== OUTPUT ===

Please wait a minute, I'll help you next. Don't worry, I'll help you find those books. Can you see the 'Sustainability' section on the bookshelf?


In [ ]:
def run_scenarios(out_dir="outputs"):
    import os, json, time
    from datetime import datetime

    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, "Outputs.jsonl")

    conditions = ["baseline", "guardrails", "guardrails_template"]
    total = 0

    with open(out_path, "w", encoding="utf-8") as f:
        for scenario_key, s in SCENARIOS.items():
            for variant_idx, user_text in enumerate(s["variants"]):
                for condition in conditions:
                    prompt = build_prompt(scenario_key, condition, variant_idx)
                    output = generate_once(prompt)

                    rec = {
                        "scenario_key": scenario_key,
                        "scenario_title": s["scenario_title"],
                        "condition": condition,
                        "variant_idx": variant_idx,
                        "user_text": user_text,
                        "prompt": prompt,
                        "output": output,
                    }

                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    total += 1

    print(f"Done. Saved {total} generations to {out_path}")
    return out_path

In [ ]:
#Running the full experiment, producing 75 outputs
#75 outputs = (5 scenarios) * (3 conditions of guardrails) * (5 variants of user input)
jsonl_file = run_scenarios()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Done. Saved 75 generations to outputs/Outputs.jsonl


In [14]:
#Exporting the experiement outputs into an excel file for further review
import json
import pandas as pd

# Load JSONL produced by run_scenarios()
records = []
with open(jsonl_file, "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

df = pd.DataFrame(records)

# Create table strucutre
review_df = df[[
    "scenario_key",
    "scenario_title",
    "condition",
    "variant_idx",
    "user_text",
    "output"
]].rename(columns={
    "scenario_key": "Scenario",
    "scenario_title": "Scenario Desc",
    "condition": "Prompt Condition",
    "variant_idx": "Variant",
    "user_text": "User Input",
    "output": "LLM Response"
})

# Add empty columns for annotation
review_df["Reviewer Notes"] = ""
review_df["Overall Appropriateness (1–5)"] = ""
review_df["Clarity (1–5)"] = ""
review_df["Educational Usefulness (1–5)"] = ""

# Export to Excel
out_path = "output_review.xlsx"
review_df.to_excel(out_path, index=False)